In [2]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

from scipy.signal import hilbert

## Import data

1 - epilepsy
0 - control

In [3]:
eeg_points: list[str] = ['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8', 'FC5', 'FC6', 'O1', 'O2', 'P7', 'P8', 'T7', 'T8']
label_encoder: dict[str, int] = {
    'epilepsy' : 1, 
    "control" : 0
}

studies: list[pd.Series] = []

In [4]:
# Guinea Bissau
meta_gb = pd.read_csv("DATASET/metadata_guineabissau.csv")

In [5]:
def get_closed_eyes_signal(signal: pd.DataFrame) -> pd.DataFrame:
    num_rows: int = len(signal)
    return signal[int(3 * num_rows / 5):]

In [6]:
for row in meta_gb.iterrows():
    subject_id = row[1]['subject.id']
    label = label_encoder[row[1].Group.lower()]
    signal = pd.read_csv(f"DATASET/EEGs_Guinea-Bissau/signal-{subject_id}.csv")
    signal = signal[eeg_points]
    
    closed_eyes_signal = get_closed_eyes_signal(signal)

    studies.append(closed_eyes_signal)

In [7]:
target = studies[0]
target_df = pd.DataFrame(target)

correlation_matrix = target_df.corr()

In [8]:
# Nigeria
meta_nig = pd.read_csv("DATASET/metadata_nigeria.csv")

In [9]:
for _, row in meta_nig.iterrows():
    subject_id = row['subject.id']
    label = label_encoder[row.Group]
    filename = row['csv.file'][:-3]
    signal = pd.read_csv(f"DATASET/EEGs_Nigeria/{filename}")
    signal = signal[eeg_points]
    
    closed_eyes_signal = get_closed_eyes_signal(signal)

    studies.append(closed_eyes_signal)

In [10]:
len(studies)

318

In [11]:
print(type(studies[0]))

<class 'pandas.core.frame.DataFrame'>


In [35]:
studies[0]

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8
AF3,1.000000,0.999824,0.999990,0.999989,0.998957,0.999899,0.999781,0.998397,0.998827,0.999983,0.999993,0.999964,0.999987,0.999973
AF4,0.999824,1.000000,0.999779,0.999777,0.998309,0.999865,0.999377,0.997286,0.998120,0.999760,0.999818,0.999820,0.999896,0.999857
F3,0.999990,0.999779,1.000000,0.999996,0.999063,0.999880,0.999833,0.998542,0.998907,0.999993,0.999993,0.999958,0.999975,0.999969
F4,0.999989,0.999777,0.999996,1.000000,0.998967,0.999898,0.999813,0.998478,0.999001,0.999999,0.999997,0.999971,0.999976,0.999978
F7,0.998957,0.998309,0.999063,0.998967,1.000000,0.998531,0.999212,0.999421,0.997464,0.998954,0.998915,0.998668,0.998837,0.998797
F8,0.999899,0.999865,0.999880,0.999898,0.998531,1.000000,0.999453,0.997716,0.998821,0.999902,0.999908,0.999882,0.999941,0.999970
FC5,0.999781,0.999377,0.999833,0.999813,0.999212,0.999453,1.000000,0.999123,0.998862,0.999808,0.999794,0.999775,0.999711,0.999675
FC6,0.998397,0.997286,0.998542,0.998478,0.999421,0.997716,0.999123,1.000000,0.998011,0.998485,0.998391,0.998218,0.998153,0.998135
O1,0.998827,0.998120,0.998907,0.999001,0.997464,0.998821,0.998862,0.998011,1.000000,0.999052,0.998922,0.998969,0.998740,0.998909
O2,0.999983,0.999760,0.999993,0.999999,0.998954,0.999902,0.999808,0.998485,0.999052,1.000000,0.999994,0.999967,0.999969,0.999978


In [19]:
def compute_plv(matrix, columns):
    """
    Compute the Phase Locking Value (PLV) for a given matrix of time-series signals.
    Parameters:
        matrix (np.ndarray): A 2D NumPy array of time-series data.
        columns (list): List of column names from the original DataFrame.
    Returns:
        pd.DataFrame: PLV matrix as a pandas DataFrame.
    """
    n_channels = matrix.shape[1]
    plv_matrix = np.zeros((n_channels, n_channels))

    phases = np.angle(hilbert(matrix, axis=0))

    for i in range(n_channels):
        for j in range(n_channels):
            phase_diff = phases[:, i] - phases[:, j]
            plv_matrix[i, j] = np.abs(np.mean(np.exp(1j * phase_diff)))

    return pd.DataFrame(plv_matrix, columns=columns, index=columns)

# Update the loop to handle this
for i, study in enumerate(studies):
    studies[i] = compute_plv(study.values, study.columns)  


In [20]:
for i, study in enumerate(studies):
    studies[i] = compute_plv(study.values, study.columns)

In [21]:
assert len(studies) == 318

In [34]:
for i, study in enumerate(studies):
    study.to_csv(f"study_{i}.csv", index=False)

In [27]:
from sklearn.model_selection import train_test_split

train_studies, temp_studies = train_test_split(studies, test_size=0.4, random_state=42)
valid_studies, test_studies = train_test_split(temp_studies, test_size=0.5, random_state=42)

print(f"Train size: {len(train_studies)}")
print(f"Validation size: {len(valid_studies)}")
print(f"Test size: {len(test_studies)}")


Train size: 190
Validation size: 64
Test size: 64


In [24]:
model = GATv2Lightning(features_shape=train_data[0].x.shape[-1]).to(device)

ModuleNotFoundError: No module named 'torch'

In [ ]:
print(torch.cuda.is_available())

In [11]:
class Args:
    timestep = 6
    epochs = 25
    batch_size = 256
    n_splits = 5
    cache_dir = "data/cache"
    exp_name = "eeg_exp"

args = Args()

In [ ]:
kfold_cval(studies, args)